In [ ]:
!nvidia-smi

In [ ]:
import sys
import os

# Add the path to the folder you want to import from
sys.path.append(os.path.abspath('../EasyEdit/'))

DEVICE_NUM = 0

MULTI = True

if not MULTI:
    os.environ["CUDA_VISIBLE_DEVICES"]= f"{DEVICE_NUM}" # '' #



In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(0)

In [ ]:
pii_type = 'pii'

In [ ]:
import pickle
from transformers import pipeline
from tqdm import tqdm
import torch
import re
from collections import defaultdict
from transformers import GPT2Tokenizer, AutoModelForCausalLM, AutoTokenizer
import random


#device = f"cuda:{DEVICE_NUM}" if torch.cuda.is_available() else 'cpu'

model_type = 'gpt-j' #'gpt-neo' #
models = ['6B'] #['1.3B', '2.7B'] #
model_size = models[0]


if model_type == 'gpt-j':
    model_name = f"EleutherAI/gpt-j-{model_size}"
elif model_type == 'gpt-neo':
    model_name = f"EleutherAI/gpt-neo-{model_size}"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [ ]:
f"{model_type}-{model_size}"

In [ ]:
import transformers
import torch
import tqdm
import pandas as pd


from datasets import load_dataset
from ast import literal_eval

In [ ]:
CONTEXT = 200

UPDATE_METHODS = ['pre_edit', f'memoedit-{CONTEXT}', f'MEMIT-{CONTEXT}']

In [ ]:
from lm_eval.models.huggingface import HFLM
from lm_eval import tasks, evaluator

In [ ]:
# Define the task you want to evaluate on
TASKS = ["hellaswag", "lambada", 'wikitext', 'winogrande', 'piqa']  
scores = {}

In [ ]:
import gc

In [ ]:
redo = False

In [ ]:
import pickle
import csv

    
def load_pickle(filename):
    with open(filename, "rb") as pickle_handler:
        results = pickle.load(pickle_handler)
    return results

In [ ]:
if not os.path.exists(f"lm-eval-harness-res-{model_type}-{model_size}-{pii_type}"):
    os.mkdir(f"lm-eval-harness-res-{model_type}-{model_size}-{pii_type}")

In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

In [ ]:
from easyeditor.models.grace.GRACE import GRACE

In [ ]:
#from easyeditor.models.grace.grace_hparams import GraceHyperParams
#from easyeditor.models.grace.GRACE import GRACEAdapter
#torch.serialization.add_safe_globals([GraceHyperParams, GRACEAdapter])
#torch.serialization.safe_globals([GraceHyperParams, GRACEAdapter])

In [ ]:
UPDATE_METHODS[-1:]

In [ ]:
scores = {}
for UPDATE_METHOD in UPDATE_METHODS:
    print('*'*80)
    print(UPDATE_METHOD)
    print('*'*80)

    filename = f'./lm-eval-harness-res-{model_type}-{model_size}-{pii_type}/{UPDATE_METHOD}.pkl'
    if not redo and os.path.exists(filename):
        print("*"*80)
        print("ATTENZIONE GIA' CALCOLATO")
        print("Generazione al momento saltata")
        print("*"*80)
        print()
        scores[UPDATE_METHOD] = load_pickle(filename)
        continue

    scores[UPDATE_METHOD] = {}  
    if UPDATE_METHOD == 'pre_edit':
        model_path = model_name
        
        model = AutoModelForCausalLM.from_pretrained(model_path)
    else:
        if UPDATE_METHOD.startswith("memoedit") or UPDATE_METHOD.startswith("MEMIT"):
            BATCH_SIZE = {'memoedit-200':8, 'MEMIT-200':8}[UPDATE_METHOD] # TODO da specificare a mano per ora
            model_path = f"../EasyEdit/edited_states_{model_type}-{model_size}/{UPDATE_METHOD.replace('-', '_')}_{BATCH_SIZE}_{pii_type}_all_edited_states.pt"
        #elif UPDATE_METHOD.startswith('dememorize'):
        #    model_path = f"../DeMemorization-main/{UPDATE_METHOD}_{model_type}-{model_size}_{pii_type}"
        else:
            model_path = f"../EasyEdit/edited_states_{model_type}-{model_size}/{UPDATE_METHOD.replace('-', '_')}_all_edited_states.pt"
        
        print(model_path)
        
        if not os.path.exists(model_path):
            print("Edited states not computed, skipped!")
            continue
            
        
        if not UPDATE_METHOD.startswith('dememorize') and not UPDATE_METHOD.startswith('GRACE'):
            model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')
            
            #model = model.to(device)
            
            edited_layes = torch.load(model_path)#, map_location='auto')#torch.device(device))
            edited_states = model.state_dict()
            
            for i in edited_layes.keys():
                edited_states[f"{i}.weight"] = edited_layes[i]
            
            model.load_state_dict(edited_states)
            del edited_states
        elif UPDATE_METHOD.startswith('GRACE'):
            model = AutoModelForCausalLM.from_pretrained(model_name)
            model = model.to(device)
            model = GRACE.from_pretrained(
                model=model,
                device=device,
                adapter_ckpt_path=model_path,
                weights_only=False
            )
            model.activate_inference_state()
            model=model.model
        
        else:
            model = AutoModelForCausalLM.from_pretrained(model_path)
            
    #model = model.to(device)
    model, tokenizer = accelerator.prepare(model, tokenizer)

    model.eval()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    display(model)
    
    # Load model wrapper
    model = HFLM(pretrained=model, 
                 tokenizer=tokenizer, batch_size=8)

    # Compute scores
    for task in TASKS:
        results = evaluator.simple_evaluate(
            model=model,
            tasks=[task],
            num_fewshot=0,
            limit=500,
            bootstrap_iters=500 if model_size!='6B' else 50,
            max_batch_size=32 if model_size!='6B' else 4
        )

        for t in results['results']: 
            scores[UPDATE_METHOD][t] = results['results'][t]
    
    model._model = model._model.to('cpu')
    del model._model
    del model
    gc.collect()
    
    torch.cuda.empty_cache()

    with open(filename, "wb") as pickle_handler:
        pickle.dump(scores[UPDATE_METHOD], pickle_handler)

In [ ]:
to_print_scores = {}
for UPDATE_METHOD in scores:
    for t in scores[UPDATE_METHOD]:
        if t not in to_print_scores:
            to_print_scores[t] = {}
        to_print_scores[t][UPDATE_METHOD] = scores[UPDATE_METHOD][t]
        to_print_scores[t][UPDATE_METHOD]['method'] = UPDATE_METHOD

to_print_scores

In [ ]:
scores = to_print_scores

In [ ]:
import pandas as pd

for t in scores:
    df = pd.DataFrame([scores[t][UPDATE_METHOD] for UPDATE_METHOD in scores[t]])
    display(df)
    df.to_csv(f'lm-eval-harness-res-{model_type}-{model_size}-{pii_type}/{t}.csv', index=None)

In [ ]:
dfs = {}
for t in scores:
    df = pd.DataFrame([scores[t][UPDATE_METHOD] for UPDATE_METHOD in scores[t]])
    df = df.rename(columns=lambda x: x if ',none' not in x else x.split(',none')[0])
    
    metric = df.columns[1]
    df[f'{metric}_stderr'] = df[f'{metric}_stderr'].replace('N/A', 0)
    
    display(df[[metric, f'{metric}_stderr', 'method']])
    dfs[t] = df[[metric, f'{metric}_stderr', 'method']]

In [ ]:
def map_method_names(values):
    new_names = []
    mapping = {
        'pre_edit': 'Pre Edit',
    f'memoedit-{CONTEXT}': 'PME', 
    f'MEMIT-{CONTEXT}': 'MEMIT', 
    f'dememorize-{CONTEXT}': 'DeMem',
    f'GRACE-{CONTEXT}': 'GRACE'
    }
    return [mapping[v] for v in values]

def rename_metric(x):
    m = {
        'acc': 'Accuracy↑',
        'perplexity': 'Perplexity↓',
        'word_perplexity': 'Word Perplexity↓'
    }
    return m[x]


In [ ]:
mapping = {
        'pre_edit': 'Pre Edit',
    f'memoedit-{CONTEXT}': 'PME', 
    f'MEMIT-{CONTEXT}': 'MEMIT', 
    f'dememorize-{CONTEXT}': 'DeMem',
    f'GRACE-{CONTEXT}': 'GRACE'
    }



to_print_scores = {}

for t in dfs:
    to_print_scores[t] = dfs[t].set_index('method')
    metric = to_print_scores[t].columns[0]
    to_print_scores[t][metric] = [f'{round(v, 2)}({round(s,2)})' for v, s in zip(to_print_scores[t][metric], to_print_scores[t][f'{metric}_stderr'])]
    to_print_scores[t] = to_print_scores[t][[metric]].T

to_print_scores = pd.concat(to_print_scores)
to_print_scores = to_print_scores.rename(columns= lambda x: mapping[x])
to_print_scores.index = [ f"{t.replace('_', ' ').capitalize()} {rename_metric(m)}"
    for t, m in to_print_scores.index
]
to_print_scores

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import to_hex, LinearSegmentedColormap



    


dataframes = []
tasks = []

for k, df in dfs.items():
    dataframes.append(df)
    tasks.append(k)

forest_green_gradient = LinearSegmentedColormap.from_list(
    "forest_green_gradient", ["#a8e6a3", "#1b5e20"]
)

fig, axes = plt.subplots(1, len(dataframes), figsize=(len(dataframes)*3, 3), sharey=False)

for i, df in enumerate(dataframes):
    ax = axes[i]
    methods = map_method_names(df['method'].values)
    metrics = df[df.columns[0]]
    stds = df[df.columns[1]]

    # Generate colors based on the number of methods
    num_methods = len(methods)
    colors = [to_hex(forest_green_gradient(j / (num_methods - 1))) for j in range(num_methods)]

    # Plot bars with gradient colors
    ax.bar(methods, metrics, capsize=5, color=colors, alpha=0.9)

    ax.set_title(f"{tasks[i].replace('_', ' ').capitalize()}", fontsize=16)

    # Custom limits and y-ticks
    y_min = (metrics - stds).min() if sum(stds)!=0 else (metrics - 0.1).min()
    y_max = (metrics + 1.2*stds).max() if sum(stds)!=0 else (metrics + 0.1).max()
    y_padding = (y_max - y_min) * 0.1

    ax.set_ylim(y_min - y_padding, y_max + y_padding)

    ax.yaxis.set_major_locator(plt.MaxNLocator(nbins=6))
    ax.set_ylabel(rename_metric(df.columns[0]), fontsize=16)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(f'lm-eval-harness-res-{model_type}-{model_size}-{pii_type}.png')
plt.show()

In [ ]:
exit()